# 文本分類任務

如前所述，我們將專注於基於 **AG_NEWS** 數據集的簡單文本分類任務，目的是將新聞標題分類為以下四個類別之一：國際、體育、商業和科學/技術。

## 數據集

此數據集已內建於 [`torchtext`](https://github.com/pytorch/text) 模組中，因此我們可以輕鬆存取它。


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

在這裡，`train_dataset` 和 `test_dataset` 包含分別返回標籤（類別編號）和文本的集合，例如：


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

所以，讓我們列印出資料集中的前10個新標題：


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

因為數據集是迭代器，如果我們想多次使用數據，我們需要將其轉換為列表：


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## 分詞

現在我們需要將文本轉換成**數字**，以便表示為張量。如果我們想要詞級表示，需要完成以下兩件事：
* 使用**分詞器**將文本拆分成**詞元**
* 建立這些詞元的**詞彙表**。


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

使用詞彙表，我們可以輕鬆地將標記化的字串編碼為一組數字：


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## 詞袋文字表示法

由於文字代表意義，有時我們可以僅透過查看個別的文字來理解文本的含義，而不需要考慮它們在句子中的順序。例如，在分類新聞時，像 *天氣*、*雪* 這些詞可能表明是 *天氣預報*，而像 *股票*、*美元* 則可能屬於 *財經新聞*。

**詞袋** (BoW) 向量表示法是最常用的傳統向量表示法之一。每個文字都與向量索引相關聯，向量元素包含某個文字在特定文件中出現的次數。

![顯示詞袋向量表示法如何在記憶體中表示的圖片。](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **注意**：你也可以將 BoW 理解為文本中每個文字的獨立一熱編碼向量的總和。

以下是一個使用 Scikit Learn Python 庫生成詞袋表示法的示例：


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

要從我們的 AG_NEWS 數據集的向量表示計算詞袋向量，可以使用以下函數：


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **注意：** 這裡我們使用全域變數 `vocab_size` 來指定詞彙表的預設大小。由於詞彙表的大小通常相當大，我們可以將詞彙表的大小限制為最常見的詞彙。嘗試降低 `vocab_size` 的值並運行以下程式碼，觀察它如何影響準確性。你應該預期會有一些準確性的下降，但不會太劇烈，以換取更高的效能。


## 訓練 BoW 分類器

現在我們已經學會如何建立文字的詞袋表示法，接下來讓我們在此基礎上訓練一個分類器。首先，我們需要將數據集轉換為適合訓練的格式，將所有位置向量表示法轉換為詞袋表示法。這可以通過將 `bowify` 函數作為標準 torch `DataLoader` 的 `collate_fn` 參數來實現：


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

現在讓我們定義一個簡單的分類器神經網絡，其中包含一個線性層。輸入向量的大小等於 `vocab_size`，輸出大小對應於類別數量（4）。由於我們正在解決分類任務，最終的激活函數是 `LogSoftmax()`。


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

現在我們將定義標準的 PyTorch 訓練循環。由於我們的數據集相當大，為了教學目的，我們只會訓練一個 epoch，有時甚至少於一個 epoch（通過指定 `epoch_size` 參數可以限制訓練）。我們還會在訓練過程中報告累積的訓練準確率；報告的頻率是通過 `report_freq` 參數指定的。


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams、TriGrams 和 N-Grams

袋裝詞語（Bag of Words）方法的一個限制是，有些詞語是多詞組成的表達，例如「hot dog」這個詞語的意思與「hot」和「dog」在其他語境中的意思完全不同。如果我們總是用相同的向量來表示「hot」和「dog」，可能會讓模型感到混淆。

為了解決這個問題，**N-gram 表示法**經常被用於文件分類的方法中，其中每個單詞、雙詞或三詞的出現頻率是訓練分類器的一個有用特徵。例如，在雙詞組（bigram）表示法中，除了原本的單詞外，我們還會將所有的詞對加入詞彙表中。

以下是一個使用 Scikit Learn 生成雙詞組袋裝詞語表示法的例子：


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram 方法的主要缺點是詞彙量會迅速增長。在實際應用中，我們需要將 N-gram 表示法與一些降維技術結合使用，例如 *嵌入*，我們會在下一單元中討論。

要在我們的 **AG News** 數據集中使用 N-gram 表示法，我們需要建立專門的 ngram 詞彙：


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


我們可以使用上述相同的程式碼來訓練分類器，但這樣會非常耗費記憶體。在下一個單元中，我們將使用嵌入來訓練二元語法分類器。

> **注意：** 你只能保留那些在文本中出現次數超過指定數量的 ngrams。這樣可以確保罕見的二元語法會被省略，並顯著減少維度。為此，將 `min_freq` 參數設置為更高的值，並觀察詞彙表長度的變化。


## 詞頻-逆文檔頻率 TF-IDF

在 BoW 表示法中，詞語的出現次數被均等地加權，無論詞語本身的特性如何。然而，很明顯像 *a*、*in* 等這些常見詞對分類的作用遠不如一些專業術語。事實上，在大多數 NLP 任務中，有些詞語比其他詞語更具相關性。

**TF-IDF** 代表 **詞頻–逆文檔頻率**。它是袋子模型（BoW）的變體，與使用二進制 0/1 值來表示詞語是否出現在文檔中的方法不同，TF-IDF 使用浮點值，該值與詞語在語料庫中的出現頻率相關。

更正式地說，詞語 $i$ 在文檔 $j$ 中的權重 $w_{ij}$ 定義如下：
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
其中：
* $tf_{ij}$ 是詞語 $i$ 在文檔 $j$ 中的出現次數，即我們之前看到的 BoW 值
* $N$ 是語料庫中的文檔總數
* $df_i$ 是包含詞語 $i$ 的文檔數量

TF-IDF 值 $w_{ij}$ 與詞語在文檔中出現的次數成正比，但會根據語料庫中包含該詞語的文檔數量進行調整，這有助於平衡某些詞語出現頻率較高的情況。例如，如果某個詞語出現在語料庫中的 *每一份* 文檔中，則 $df_i=N$，而 $w_{ij}=0$，這些詞語將被完全忽略。

你可以使用 Scikit Learn 輕鬆地創建文本的 TF-IDF 向量化：


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## 結論

然而，儘管 TF-IDF 表示法為不同的詞語提供了頻率權重，但它無法表達詞語的意義或順序。正如著名語言學家 J. R. Firth 在1935年所說：「詞語的完整意義總是與上下文相關，任何脫離上下文的意義研究都不應被認真對待。」我們稍後會在課程中學習如何通過語言建模從文本中捕捉上下文信息。



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原文文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
